In [1]:
import numpy as np 
import pandas as pd
from PIL import Image
import os
import json

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [3]:
BATCH = 256
EPOCHS = 8

LR = 0.01
IM_SIZE = 224

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TRAIN_DIR = '../input/herbarium-2021-fgvc8/train/'
TEST_DIR = '../input/herbarium-2021-fgvc8/test/'

In [4]:
%%time

with open(TRAIN_DIR + 'metadata.json', "r", encoding="ISO-8859-1") as file:
    train = json.load(file)    

CPU times: user 6.76 s, sys: 1.38 s, total: 8.14 s
Wall time: 11.5 s


In [5]:
train_img = pd.DataFrame(train['images'])
train_ann = pd.DataFrame(train['annotations']).drop(columns='image_id')
train_df = train_img.merge(train_ann, on='id')

print(len(train_df))
train_df.head()

2257759


,file_name,height,id,license,width,category_id,institution_id
0,images/604/92/1814367.jpg,1000,1814367,0,678,60492,0
1,images/108/24/1308257.jpg,1000,1308257,0,666,10824,0
2,images/330/76/1270453.jpg,1000,1270453,0,739,33076,3
3,images/247/99/1123834.jpg,1000,1123834,0,672,24799,0
4,images/170/18/1042410.jpg,1000,1042410,0,675,17018,0


In [6]:
NUM_CL = len(train_df['category_id'].value_counts())
NUM_CL
# 64500

64500

In [7]:
# Just for speed up training
#tr_df = train_df[:1000]
tr_df = train_df
len(tr_df)

1000

In [8]:
X_Train, Y_Train = tr_df['file_name'].values, tr_df['category_id'].values

In [9]:
Transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [10]:
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
    
        if "train" in self.dir:             
            return self.transform(x), self.labels[index]
        elif "test" in self.dir:            
            return self.transform(x), self.fnames[index]

In [12]:
trainset = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True)

In [13]:
next(iter(trainloader))[0].shape

torch.Size([16, 3, 32, 32])

In [16]:
model = torchvision.models.resnext101_32x8d()
model.fc = nn.Linear(model.fc.in_features, NUM_CL, bias=True)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [17]:
%%time

for epoch in range(EPOCHS):
    tr_loss = 0.0

    model = model.train()

    for i, (images, labels) in enumerate(trainloader):        
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)       
        logits = model(images.float())       
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss += loss.detach().item()
    
    model.eval()
    print('Epoch: %d | Loss: %.4f'%(epoch, tr_loss / i))

KeyboardInterrupt: 

In [ ]:
%%time

with open(TEST_DIR + 'metadata.json', "r", encoding="ISO-8859-1") as file:
    test = json.load(file)

In [ ]:
test_df = pd.DataFrame(test['images'])
print(len(test_df))
test_df.head()

In [ ]:
# !!! Just for speed up experiments
# tst_df = test_df[:10]
# X_Test = tst_df['file_name'].values

X_Test = test_df['file_name'].values

In [ ]:
testset = GetData(TEST_DIR, X_Test, None, Transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False)

In [ ]:
%%time

s_ls = []

with torch.no_grad():
    model.eval()
    for image, fname in testloader: 
        image = image.to(DEVICE)
        
        logits = model(image)        
        ps = torch.exp(logits)        
        _, top_class = ps.topk(1, dim=1)
        
        for pred in top_class:
            s_ls.append([fname[0].split('/')[-1][:-4], pred.item()])

In [ ]:
sub = pd.DataFrame.from_records(s_ls, columns=['Id', 'Predicted'])
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)